In [1]:
#!import config/Settings.cs

In [2]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta6"

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta6

In [3]:
using Microsoft.SemanticKernel;

var builder = new KernelBuilder();

In [4]:
using Kernel = Microsoft.SemanticKernel.Kernel;

var (model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
var deploymentName = "practical-task-azure-ai";

builder.WithAzureOpenAIChatCompletionService(deploymentName, azureEndpoint, apiKey);

var kernel = builder.Build();

In [5]:
using Microsoft.SemanticKernel.TemplateEngine;
using Microsoft.SemanticKernel.TemplateEngine.Basic;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

var aiRequestSettings = new OpenAIRequestSettings 
{
    MaxTokens = 500,
    Temperature = 0.2,
    TopP = 0.0
};

In [6]:
var input = "I want to find top-10 books about world history";

In [7]:
// Chat with the Bot
// Create a simple prompt for the semantic function

string skPrompt = @"ChatBot: How can I help you?
User: {{$input}}

---------------------------------------------

The intent of the user in 25 words or less: ";

var getShortIntentFunction  = kernel.CreateSemanticFunction(skPrompt, aiRequestSettings, "GetRecipe");

var intentResult = await kernel.RunAsync(input, getShortIntentFunction);

Console.WriteLine(intentResult);

Error: Microsoft.SemanticKernel.Diagnostics.HttpOperationException: Service request failed.
Status: 403 (Forbidden)

Headers:
Date: Thu, 07 Dec 2023 13:31:50 GMT
Connection: keep-alive
Strict-Transport-Security: REDACTED
Content-Length: 20

 ---> Azure.RequestFailedException: Service request failed.
Status: 403 (Forbidden)

Headers:
Date: Thu, 07 Dec 2023 13:31:50 GMT
Connection: keep-alive
Strict-Transport-Security: REDACTED
Content-Length: 20

   at Azure.Core.HttpPipelineExtensions.ProcessMessageAsync(HttpPipeline pipeline, HttpMessage message, RequestContext requestContext, CancellationToken cancellationToken)
   at Azure.AI.OpenAI.OpenAIClient.GetChatCompletionsAsync(String deploymentOrModelName, ChatCompletionsOptions chatCompletionsOptions, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.Connectors.AI.OpenAI.AzureSdk.ClientBase.RunRequestAsync[T](Func`1 request)
   --- End of inner exception stack trace ---
   at Microsoft.SemanticKernel

In [8]:
skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give detailed answer or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

In [9]:
// Chat with the Bot

aiRequestSettings = new OpenAIRequestSettings
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

var chatFunction = kernel.CreateSemanticFunction(skPrompt, aiRequestSettings);
var context = kernel.CreateNewContext();

var history = "";
context.Variables["history"] = history;

var userInput = "Hi, I'm looking for suggestions of best sellers about world history";
context.Variables["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(context);

Error: Microsoft.SemanticKernel.Diagnostics.HttpOperationException: Service request failed.
Status: 403 (Forbidden)

Headers:
Date: Thu, 07 Dec 2023 13:31:50 GMT
Connection: keep-alive
Strict-Transport-Security: REDACTED
Content-Length: 20

 ---> Azure.RequestFailedException: Service request failed.
Status: 403 (Forbidden)

Headers:
Date: Thu, 07 Dec 2023 13:31:50 GMT
Connection: keep-alive
Strict-Transport-Security: REDACTED
Content-Length: 20

   at Azure.Core.HttpPipelineExtensions.ProcessMessageAsync(HttpPipeline pipeline, HttpMessage message, RequestContext requestContext, CancellationToken cancellationToken)
   at Azure.AI.OpenAI.OpenAIClient.GetChatCompletionsAsync(String deploymentOrModelName, ChatCompletionsOptions chatCompletionsOptions, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.Connectors.AI.OpenAI.AzureSdk.ClientBase.RunRequestAsync[T](Func`1 request)
   --- End of inner exception stack trace ---
   at Microsoft.SemanticKernel

In [10]:
// Update the history with the output and set this as the new input value for the next request

history += $"\nUser: {userInput}\nSuggestions: {bot_answer.GetValue<string>()}\n";
context.Variables.Update(history);

Console.WriteLine(context);

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#11.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [11]:
// Chatting more)

Func<string, Task> Chat = async (string input) => {
    // Save new message in the context variables
    context.Variables["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(context);

    // Append the new interaction to the chat history
    history += $"\nUser: {input}\nSuggestions: {answer.GetValue<string>()}\n"; 
    context.Variables["history"] = history;
    
    // Show the response
    Console.WriteLine(context);
};

In [12]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");

Error: Microsoft.SemanticKernel.Diagnostics.HttpOperationException: Service request failed.
Status: 403 (Forbidden)

Headers:
Date: Thu, 07 Dec 2023 13:31:51 GMT
Connection: keep-alive
Strict-Transport-Security: REDACTED
Content-Length: 20

 ---> Azure.RequestFailedException: Service request failed.
Status: 403 (Forbidden)

Headers:
Date: Thu, 07 Dec 2023 13:31:51 GMT
Connection: keep-alive
Strict-Transport-Security: REDACTED
Content-Length: 20

   at Azure.Core.HttpPipelineExtensions.ProcessMessageAsync(HttpPipeline pipeline, HttpMessage message, RequestContext requestContext, CancellationToken cancellationToken)
   at Azure.AI.OpenAI.OpenAIClient.GetChatCompletionsAsync(String deploymentOrModelName, ChatCompletionsOptions chatCompletionsOptions, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.Connectors.AI.OpenAI.AzureSdk.ClientBase.RunRequestAsync[T](Func`1 request)
   --- End of inner exception stack trace ---
   at Microsoft.SemanticKernel

In [13]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");

Error: Microsoft.SemanticKernel.Diagnostics.HttpOperationException: Service request failed.
Status: 403 (Forbidden)

Headers:
Date: Thu, 07 Dec 2023 13:31:51 GMT
Connection: keep-alive
Strict-Transport-Security: REDACTED
Content-Length: 20

 ---> Azure.RequestFailedException: Service request failed.
Status: 403 (Forbidden)

Headers:
Date: Thu, 07 Dec 2023 13:31:51 GMT
Connection: keep-alive
Strict-Transport-Security: REDACTED
Content-Length: 20

   at Azure.Core.HttpPipelineExtensions.ProcessMessageAsync(HttpPipeline pipeline, HttpMessage message, RequestContext requestContext, CancellationToken cancellationToken)
   at Azure.AI.OpenAI.OpenAIClient.GetChatCompletionsAsync(String deploymentOrModelName, ChatCompletionsOptions chatCompletionsOptions, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.Connectors.AI.OpenAI.AzureSdk.ClientBase.RunRequestAsync[T](Func`1 request)
   --- End of inner exception stack trace ---
   at Microsoft.SemanticKernel

In [14]:
await Chat("Which topic from the ones you listed do you think the most popular?");

Error: Microsoft.SemanticKernel.Diagnostics.HttpOperationException: Service request failed.
Status: 403 (Forbidden)

Headers:
Date: Thu, 07 Dec 2023 13:31:51 GMT
Connection: keep-alive
Strict-Transport-Security: REDACTED
Content-Length: 20

 ---> Azure.RequestFailedException: Service request failed.
Status: 403 (Forbidden)

Headers:
Date: Thu, 07 Dec 2023 13:31:51 GMT
Connection: keep-alive
Strict-Transport-Security: REDACTED
Content-Length: 20

   at Azure.Core.HttpPipelineExtensions.ProcessMessageAsync(HttpPipeline pipeline, HttpMessage message, RequestContext requestContext, CancellationToken cancellationToken)
   at Azure.AI.OpenAI.OpenAIClient.GetChatCompletionsAsync(String deploymentOrModelName, ChatCompletionsOptions chatCompletionsOptions, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.Connectors.AI.OpenAI.AzureSdk.ClientBase.RunRequestAsync[T](Func`1 request)
   --- End of inner exception stack trace ---
   at Microsoft.SemanticKernel

In [15]:
Console.WriteLine(context.Variables["history"]);

In [16]:
Console.WriteLine(context.Variables["userInput"]);

Which topic from the ones you listed do you think the most popular?
